
# 🫀 Heart Disease Prediction – Training Pipeline (Complete Notebook)

## Project Overview
This project builds **two machine learning models** to predict the **10-year risk of Coronary Heart Disease (CHD)** using the *Framingham Heart Study* dataset.

### Why two models?
- **Model A**: Uses **all available features** → higher accuracy, backend/clinical usage
- **Model B**: Uses **limited, easy-to-collect features** → lightweight, fast, frontend/mobile usage

### Key Techniques Used
- XGBoost (GPU accelerated)
- Stratified Train/Test Split
- Class imbalance handling
- Hyperparameter tuning (RandomizedSearchCV)
- Probability calibration (Sigmoid / Platt Scaling)
- Custom threshold optimization (Recall-focused)



## 1️⃣ Import Libraries
We import:
- Data handling libraries (NumPy, Pandas)
- Scikit-learn tools for modeling, evaluation, calibration
- XGBoost for high-performance gradient boosting
- SciPy for hyperparameter distributions


In [1]:

import numpy as np
import pandas as pd
import joblib

from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    roc_auc_score,
    average_precision_score,
    recall_score
)
from sklearn.calibration import CalibratedClassifierCV

from xgboost import XGBClassifier
from scipy.stats import randint, uniform, loguniform



## 2️⃣ Load Dataset
We load the Framingham Heart Study dataset.
Target column:
- **TenYearCHD** → 1 = Heart disease within 10 years, 0 = No disease


In [2]:

# using  github raw  link
url = "https://raw.githubusercontent.com/chetan-j123/Heart_decision_prediction_project/master/framingham_heart_study.csv"
data = pd.read_csv(url)
print(data.head())

   male  age  education  currentSmoker  cigsPerDay  BPMeds  prevalentStroke  \
0     1   39        4.0              0         0.0     0.0                0   
1     0   46        2.0              0         0.0     0.0                0   
2     1   48        1.0              1        20.0     0.0                0   
3     0   61        3.0              1        30.0     0.0                0   
4     0   46        3.0              1        23.0     0.0                0   

   prevalentHyp  diabetes  totChol  sysBP  diaBP    BMI  heartRate  glucose  \
0             0         0    195.0  106.0   70.0  26.97       80.0     77.0   
1             0         0    250.0  121.0   81.0  28.73       95.0     76.0   
2             0         0    245.0  127.5   80.0  25.34       75.0     70.0   
3             1         0    225.0  150.0   95.0  28.58       65.0    103.0   
4             0         0    285.0  130.0   84.0  23.10       85.0     85.0   

   TenYearCHD  
0           0  
1           0  
2 


## 3️⃣ Handle Missing Values
Medical datasets often contain missing values.
We use:
- **Median** for continuous variables (robust to outliers)
- **Mode** for binary/categorical features


In [ ]:

for col in ["education", "cigsPerDay", "totChol", "BMI", "glucose", "heartRate"]:
    data[col] = data[col].fillna(data[col].median())

data["BPMeds"] = data["BPMeds"].fillna(data["BPMeds"].mode()[0])



## 4️⃣ Feature / Target Split
- **X** → Input features
- **y** → Target (TenYearCHD)


In [ ]:

X = data.drop(columns=["TenYearCHD"])
y = data["TenYearCHD"]



## 5️⃣ Train-Test Split (Stratified)
We use **stratification** to preserve class distribution
because heart disease data is imbalanced.


In [ ]:

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.25,
    stratify=y,
    random_state=42
)



## 6️⃣ Handle Class Imbalance
XGBoost supports imbalance via `scale_pos_weight`.
This prevents the model from ignoring minority (disease) cases.


In [ ]:

scale_pos_weight = (y_train == 0).sum() / (y_train == 1).sum()
scale_pos_weight



## 7️⃣ Hyperparameter Search – Model A
We use **RandomizedSearchCV** instead of GridSearch:
- Faster
- Better coverage of parameter space

Scoring metric: **F1-score**


In [ ]:

param_grid = {
    "n_estimators": randint(400, 1000),
    "learning_rate": loguniform(0.01, 0.06),
    "max_depth": randint(3, 6),
    "min_child_weight": randint(4, 15),
    "gamma": uniform(0.3, 2.0),
    "subsample": uniform(0.6, 0.25),
    "colsample_bytree": uniform(0.6, 0.25),
    "reg_lambda": loguniform(0.8, 6.0),
}

grid = RandomizedSearchCV(
    estimator=XGBClassifier(
        objective="binary:logistic",
        eval_metric="aucpr",
        scale_pos_weight=scale_pos_weight,
        random_state=42,
        tree_method="hist",
        device="cuda",
        predictor="gpu_predictor",
        n_jobs=-1
    ),
    param_distributions=param_grid,
    scoring="f1",
    n_iter=100,
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
    refit=True,
    verbose=2,
    n_jobs=-1
)

grid.fit(X_train, y_train)



## 8️⃣ Probability Calibration (Model A)
Why calibration?
- Raw ML probabilities are often **overconfident**
- Medical decisions need **reliable probabilities**

We use **Sigmoid Calibration (Platt Scaling)** with CV.


In [ ]:

best_xgb = grid.best_estimator_

model_A = CalibratedClassifierCV(
    estimator=best_xgb,
    method="sigmoid",
    cv=5
)

model_A.fit(X_train, y_train)



## 9️⃣ Threshold Optimization
Default threshold (0.5) is not optimal for healthcare.
We sweep thresholds to achieve **Recall ≥ 75%**.


In [ ]:

y_proba = model_A.predict_proba(X_test)[:, 1]

best_threshold = None
for t in np.arange(0.10, 0.51, 0.05):
    y_tmp = (y_proba >= t).astype(int)
    r = recall_score(y_test, y_tmp)
    print(f"threshold={t:.2f} | recall={r:.3f}")
    if r >= 0.75 and best_threshold is None:
        best_threshold = t

if best_threshold is None:
    best_threshold = 0.25

best_threshold



## 🔟 Final Evaluation – Model A
We evaluate using:
- Confusion Matrix
- Classification Report
- ROC-AUC
- PR-AUC


In [ ]:

y_pred = (y_proba >= best_threshold).astype(int)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_proba))
print("PR-AUC :", average_precision_score(y_test, y_proba))



## 1️⃣1️⃣ Model B – Lightweight Model
Uses fewer features for:
- Faster inference
- Easier data collection


In [ ]:

MODEL_B_FEATURES = [
    "age", "male", "BMI", "sysBP",
    "diaBP", "totChol", "glucose", "currentSmoker"
]

X_b = data[MODEL_B_FEATURES]
y_b = data["TenYearCHD"]

Xb_train, Xb_test, yb_train, yb_test = train_test_split(
    X_b, y_b,
    test_size=0.25,
    stratify=y_b,
    random_state=42
)



## 1️⃣2️⃣ Train, Calibrate & Evaluate Model B
Same pipeline, fewer features.


In [ ]:

scale_pos_weight_b = (yb_train == 0).sum() / (yb_train == 1).sum()

random_search_b = RandomizedSearchCV(
    estimator=XGBClassifier(
        objective="binary:logistic",
        eval_metric="aucpr",
        scale_pos_weight=scale_pos_weight_b,
        random_state=42,
        tree_method="hist",
        device="cuda",
        predictor="gpu_predictor",
        n_jobs=-1
    ),
    param_distributions={
        "n_estimators": randint(100, 500),
        "learning_rate": uniform(0.01, 0.2),
        "max_depth": randint(3, 6)
    },
    n_iter=50,
    scoring="f1",
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
    verbose=1,
    refit=True,
    n_jobs=-1
)

random_search_b.fit(Xb_train, yb_train)

model_b = CalibratedClassifierCV(
    estimator=random_search_b.best_estimator_,
    method="sigmoid",
    cv=5
)

model_b.fit(Xb_train, yb_train)

yb_proba = model_b.predict_proba(Xb_test)[:, 1]
yb_pred = (yb_proba >= 0.25).astype(int)

print(confusion_matrix(yb_test, yb_pred))
print(classification_report(yb_test, yb_pred))


NameError: name 'yb_train' is not defined


## 1️⃣3️⃣ Save Models & Metadata
Saved artifacts:
- model_A.pkl / model_B.pkl
- feature lists
- decision thresholds


In [ ]:

joblib.dump(model_A, "model_A.pkl")
joblib.dump(model_b, "model_B.pkl")
joblib.dump(X.columns.tolist(), "model_a_features.pkl")
joblib.dump(MODEL_B_FEATURES, "model_b_features.pkl")
joblib.dump(best_threshold, "model_A_threshold.pkl")
joblib.dump(0.25, "model_B_threshold.pkl")
